## ReAct web search

- 외부 API 를 어떻게 툴로 호출하는지 보여주는 예제

In [17]:
import os
import sys
import logging

from dotenv import load_dotenv
from llama_index.core.agent import ReActAgent
from llama_index.core.tools.types import BaseTool
from llama_index.tools.tavily_research import TavilyToolSpec
from llama_index.llms.bedrock_converse import BedrockConverse

In [18]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

load_dotenv()

True

In [19]:
MODEL_ID=os.environ.get("MODEL_ID", "anthropic.claude-3-haiku-20240307-v1:0")
AWS_PROFILE_NAME=os.environ.get("AWS_PROFILE_NAME", None)
TAVILY_API_KEY=os.environ.get("TAVILY_API_KEY", None)

assert TAVILY_API_KEY

## Setup Tools

- 웹 검색을 위해 [Tavily](https://app.tavily.com/) 서비스의 API 를 [Tool](https://llamahub.ai/l/tools/llama-index-tools-tavily-research) 로 사용

In [21]:
# setup tools
tools: list[BaseTool] = [
    TavilyToolSpec(api_key=TAVILY_API_KEY).to_tool_list()[0],
]

In [27]:
print(tools[0].metadata.description)

search(query: str, max_results: Optional[int] = 6) -> List[llama_index.core.schema.Document]

        Run query through Tavily Search and return metadata.

        Args:
            query: The query to search for.
            max_results: The maximum number of results to return.

        Returns:
            results: A list of dictionaries containing the results:
                url: The url of the result.
                content: The content of the result.

        


## Setup Agent

In [ ]:
# setup llm
llm = BedrockConverse(
    model=MODEL_ID,
    max_tokens=1024 * 2,
    temperature=0.4,
    profile_name=AWS_PROFILE_NAME,
)

In [11]:
# setup agent
agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
)

## Invoke agent with tools

In [12]:
resp = agent.chat(
    "NCSoft 의 최신작과 월드 오브 워크래프트의 차이가 뭐야?"
    "캐주얼한 게이머에게 어떤 게임이 더 좋을까?"
)

> Running step a6c8bacd-0308-4b98-90e0-45e422a461e8. Step input: NCSoft 의 최신작과 월드 오브 워크래프트의 차이가 뭐야?캐주얼한 게이머에게 어떤 게임이 더 좋을까?
Thought: The current language of the user is: Korean. I need to use a tool to help me answer the question.
Action: search
Action Input: {'query': 'NCSoft 최신작 월드 오브 워크래프트 차이', 'max_results': 6}
Observation: [Document(id_='35217ca2-0999-4635-a4ed-1f4607e86326', embedding=None, metadata={'url': 'https://m.ebn.co.kr/news/view/998690/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='월드 오브 워크래프트(이하 와우) 클래식이 발매됐다. 이에 기존작과 달라진 점을 알아봤다. 27일 와우 오리지널 리마스터 버전인 mmoprg 게임 와우 클래식이 국내에 공개됐다. 지난 2004년 처음 모습을 드러낸 와우는 10년 만에 클래식이라는 이름으로 돌아오면서 시선을 모으고 있다.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='e935428c-9e8d-433b-bc19-b2e0e5c75417', embedding=None, metadata={'url': 'https://www.etoday.co.kr/news/vie